In [6]:
import tangelo
from tangelo import SecondQuantizedMolecule

# LiH = [('Li', (0, 0, 0)), ('H', (0, 0, 1.5949))]
# diatomic_bond_length = 1.45
diatomic_bond_length = 1.5949
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3
# Set Hamiltonian parameters for LiH simulation in active space.
# diatomic_bond_length = 1.45
# geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
# basis = 'sto-3g'
# multiplicity = 1
# active_space_start = 1
# active_space_stop = 3
mol_LiH = SecondQuantizedMolecule(geometry, q=0, spin=0, basis="sto-3g")
# mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="sto-3g")

print(mol_LiH)


SecondQuantizedMolecule(xyz=[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.5949))], q=0, spin=0, solver=<tangelo.toolboxes.molecular_computation.integral_solver_pyscf.IntegralSolverPySCF object at 0x31a8297c0>, n_atoms=2, n_electrons=4, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-7.862026959394137, mo_energies=array([-2.34864417, -0.28570476,  0.07826185,  0.16393842,  0.16393842,
        0.54912925]), mo_occ=array([2., 2., 0., 0., 0., 0.]), n_mos=6, n_sos=12, active_occupied=[1], frozen_occupied=[0], active_virtual=[2, 3, 4, 5], frozen_virtual=[])


In [7]:
from tangelo.algorithms import FCISolver, CCSDSolver, VQESolver

fci_solver = FCISolver(mol_LiH)
energy_fci = fci_solver.simulate()


print(f"FCI energy:\t{energy_fci}")


FCI energy:	-7.882175990801266


In [8]:
vqe_solver = VQESolver({
    'molecule': mol_LiH,
    # 'verbose': True,
})
vqe_solver.build()
energy_vqe = vqe_solver.simulate()
print(f"VQE energy:\t{energy_vqe}")

VQE energy:	-7.882174843682616


In [10]:
import cirq
import openfermion
from openfermion.circuits import trotter

molecule = openfermion.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))

# print(openfermion.get_fermion_operator(molecular_hamiltonian))

time = 1.
final_rank = 2

# Initialize circuit qubits in a line.
n_qubits = openfermion.count_qubits(molecular_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)

# Compile the low rank Trotter step using OpenFermion.
custom_algorithm = trotter.LowRankTrotterAlgorithm(final_rank=final_rank)
circuit = cirq.Circuit(
    trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, omit_final_swaps=True,
            algorithm=custom_algorithm),
    strategy=cirq.InsertStrategy.EARLIEST)

# Print circuit.
cirq.drop_negligible_operations(circuit)
print(circuit.to_text_diagram(transpose=True))


0             1                    2                3
│             │                    │                │
Rz(π)         Rz(π)                Rz(π)            Rz(π)
│             │                    │                │
│             PhISwap(0.25)────────PhISwap(0.25)^-1 │
│             │                    │                │
PhISwap(0.25)─PhISwap(0.25)^0.081  Z^0              │
│             │                    │                │
Rz(0.337π)    Z^0                  PhISwap(0.25)────PhISwap(0.25)^-0.081
│             │                    │                │
Rz(0)         PhISwap(0.25)────────PhISwap(0.25)^-1 Z^0
│             │                    │                │
│             Rz(0.337π)           Z^0              Rz(0.169π)
│             │                    │                │
│             Rz(0)                Rz(0.169π)       Rz(0)
│             │                    │                │
│             │                    Rz(0)            │
│             │                    │        

In [ ]:
from tangelo.toolboxes.ansatz_generator.ansatz_utils import controlled_pauliwords, trotterize

qubit_hamiltonian = vqe_solver.qubit_hamiltonian
tau = 0.04

# Controlled time-evolution of qu_op
c_trott = trotterize(qubit_hamiltonian, time=tau, n_trotter_steps=1, trotter_order=1) # , control=4)

In [5]:
# def try_cr():
# cr_vqe_solver = VQESolver({
#     'molecule': SecondQuantizedMolecule(
#         [('Cr', (0, 0, 0))], q=0, spin=0, basis="sto-3g",
#         frozen_orbitals=4
#     ),
#     'verbose': True,
    
# })
# cr_vqe_solver.build()


In [6]:
# cr_energy_vqe = cr_vqe_solver.simulate()
# print(f"VQE energy:\t{cr_energy_vqe}")